# **Domain Adaptation to train model-independent classifiers in High Energy Physics**

Training Machine Learning classifiers to distinguish signal from background in High Energy Physics is usually based on simulated datasets, at least for the signal component. The simulation of signal relies on theoretical models that often include hypotheses that have not been completely validated, yet. As a consequence, an implicit bias is introduced in the classifier performance that will differ from model to model.

Domain adaptation can be used to mitigate the dependence of the trained classifier on the theoretical model adopted for training, forcing the classifier Neural Network to actively ignore the information necessary to distinguish different theoretical models.

In this exercise, which is inspired by a recently published study ([EPJC 82 (2022)](https://link.springer.com/article/10.1140/epjc/s10052-022-10871-3), [arXiv:2207.09293](https://arxiv.org/abs/2207.09293)), we will implement a simplified version of the method to simulated LHC datasets, discussing the most interesting features and pitfalls of the method.

# Brief description of the physics use case

We consider the physics use case of a cross section measurement at the LHC. In particular the process we are interested to measure is the Higgs boson produced through *Vector Boson Fusion* (**VBF**) and decaying to a pair of W bosons. The final state of interest is the fully leptonic one, i.e. when both W bosons decay to leptons $W^\pm\to\ell^\pm\nu$.

The Feynman diagram for the signal process is depicted below:

<img src="http://lviliani.web.cern.ch/lviliani/FiguresForDANotebook/vbf.png" width="400" height="400" />

This is a rare process and the usage of Machine Learning techniques for the event classification and discrimination from backgrounds is extremely important. In this use case we consider two distinct sources of background processes: the Higgs boson production through gluon fusion and consequent decay to $WW\to2\ell2\nu$ (**ggH**), and all other sources of backgrounds (**BKG**), mainly arising from $t\bar{t}$ and non-resonant $WW$ production. Examples of the Feynman diagrams for ggH (left), $t\bar{t}$ (center) and $WW$ (right) processes are shown below.

<img src="http://lviliani.web.cern.ch/lviliani/FiguresForDANotebook/ggH.png" width="400" height="400" /> <img src="http://lviliani.web.cern.ch/lviliani/FiguresForDANotebook/ttbar.png" width="400" height="400" /> <img  src="http://lviliani.web.cern.ch/lviliani/FiguresForDANotebook/WW.png" width="400" height="400" /> 

We want to build a three-classes feed-forward Deep Neural Network to categorize the events in the VBF, ggH and BKG classes with the maximum achievable accuracy. At the same time, we want the output probability distributions to be independent of the particular VBF theoretical model used in the training and in the analysis.

# Load, read and understand the data

Let's start the hackathon by reading the dataset and understanding what we will work with.

The dataset has been already pre-processed and is provided as a *pandas* dataframe saved in a *pickle* file.
It can be loaded easily using the pandas `read_pickle` function as follows.

Let's read and print to screen the dataframe content.

In [ ]:
import pandas as pd


# IF USING COLAB, Uncomment and run the next lines and comment the next block; OTHERWISE leave all as is
#%pip install pickle5
#import pickle5 as pickle
#!rm -f dataset_DA.pkl
#!wget https://pandora.infn.it/public/488317/dl/dataset_DA.pkl
#with open('dataset_DA.pkl', "rb") as fh:
#  df = pickle.load(fh)
# END COLAB BLOCK


# IF NOT USING COLAB, please use the standard code below
df = pd.read_pickle('https://pandora.infn.it/public/488317/dl/dataset_DA.pkl')
# END NON COLAB BLOCK

pd.set_option('display.max_columns', None)
df

The dataframe contains a total of ~60k events. 

The first 24 columns contain the input features that we will use in our DNN architecture.
Although the physical meaning of all the variables is not important to understand this excercise, we still report below a brief description of each one for reference.

In general, they are high-level variables regarding leptons and jets kinematics that are known to be powerful for the discrimination of signal and background processes.

- `ptj1` and `ptj2`: the magnitudes of the transverse momenta of the leading and subleading jet;
- `etaj1` and `etaj2`: the pseudorapidity of the leading and subleading jet. Jets arising from VBF processes tend to be emitted at larger pseudorapidity with respect to background events;
- `detajj`: the separation in pseudorapidity between the two leading jets in the final state. The signal shows a larger pseudorapidity gap with respect to the backgrounds;
- `mjj`: the invariant mass of the dijet system. It has a good discrimination power since it is typically larger for the signal than for the background;
- `ptll`, `ptl1`, `ptl2`: the magnitudes of the transverse momenta of the dilepton system, the leading lepton, and the subleading lepton, respectively;
- `etal1` and `etal2`: the pseudorapidity of the leading and subleading lepton, respectively;
- `mll`: the invariant mass of the lepton pair. Due to the spin correlation effect in the $H\to WW\to2\ell2\nu$ decay chain, this variable is peaked at low values for the VBF and ggH mechanisms, while showing a broadened shape for non-resonant events;
- `dphill`: the angular separation in $\phi$ between the two leptons;
- `drll`: the radial separation between the two leptons;
- `mlj(...)`: the invariant mass of the system consisting of the i-th lepton and j-th jet (all four permutations are considered);
- `ctot`: $Ctot = \log ( \sum_\ell (2\eta_\ell - \sum_j \eta_j) / |\Delta\eta_{jj}|)$ is called centrality of the dilepton system;
- `met`: the missing transverse energy in the event;
- `mth`: the transverse mass of the system computed as $m_T^H = \sqrt{2 p_T^{\ell\ell}p_T^{miss} \cos(\Delta\phi(p_T^{\ell\ell},p_T^{miss}))}$;
- `mti`: the visible mass computed as $m_T^{vis} = \sqrt{ (p^{\ell\ell} + p_T^{miss})^2 - (\vec{p}^{\ell\ell} + \vec{p}_T^{miss})^2 }$
- `dphillmet`: the azimuthal opening angle between the dilepton system and MET, i.e. $\Delta\phi(p_T^{\ell\ell},p_T^{miss})$
- `ht`: the scalar sum of the transverse momenta of all jets in the event.

The remaining columns in the dataframe represent the labels that identify each process:
- `isVBF`: = 1 for VBF (either SM or BSM) events;
- `isBKG`: = 1 for background events;
- `isGGH`: = 1 for gluon fusion events;
- `isSM`: = 1 for SM VBF events;
- `isBSM{i}`: we included six (i = 0,...,5) different BSM models in the dataset, corresponding to various anomalous couplings of the Higgs boson with vector bosons.

The first three labels identify our **source domain ($\mathcal{S}$)**, while the latters identify the **target domain ($\mathcal{T}$)**.

## How many events of each kind do we have?

Let's know check how many events of each process we have in our dataset. In this case we opted for a balanced dataset composed by **VBF**, **ggH** and **BKG** processes in equal proportions.

Note that the data labelled as `isVBF` are composed in equal proportions of the different VBF models. In our dataset we have $2828$ events for each VBF model (either the SM and the 6 BSM models), for a total of $2828 \cdot 6 = 19796$ `isVBF` events.

In [ ]:
for col in df.columns[24:35]:
    print (col, len(df[df[col]==1]))

## Compare input features for the various processes

Let's have a deeper look at our dataset by plotting the distributions of all the input features for VBF, ggH and BKG events.

What are the most discriminating features (by eye)?

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import numpy as np
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 150

# List of features - the first 24 columns of our dataset
features = df.columns[:24]
# VBF, ggH and BKG labels
labels = df.columns[24:27]

**YOUR TURN: Plot the distributions of each feature comparing the VBF, ggH and BKG labels**

In [ ]:
### YOUR CODE HERE

Let's now plot the distributions of the input features comnparing the various VBF models.

Do you spot any particular feature that is significantly different for the different models?

**YOUR TURN: Plot the distributions of each feature comparing the isSM, isBSM0, ..., isBSM5 labels**

In [ ]:
### YOUR CODE HERE

The simplest way to achieve our goal, i.e. have a good discrimination among VBF, ggH and BKG, but at the same time being independent of the different VBF models, would be to drop the features that are sensitive to the VBF modelling but do not provide a significant discrimination among VBF, ggH and BKG.

But this is not always possible (as in the current use case), because some of the features that are good for the classification might also be good for distinguishing different models. Dropping those would mean a huge loss of classification power.

# Define the Domain Adaptation model

In order to achieve our goal we will adopt a particular flavor of Domain Adaptation, whose objective is to find a common representation for the source and target domains.

We want to build our Domain Adaptation model according to an Adversarial Deep Neural Network approach (also known as domain adversarial training), according to the following scheme:
<img src="http://lviliani.web.cern.ch/lviliani/FiguresForDANotebook/ADNN.png" width="800" height="800" />

The main components of the ADNN are:
- the *Classifier (C)*: a feed-forward Deep Neural Network with the goal to classify events in 3 classes (VBF signal, ggH and BKG);
- the *Adversary (A)*: a feed-forward Deep Neural Network connected to the second-to-last layer of C (representation) with the goal to classify different signal models.

*C* and *A* are trained in a competitive way with the final goal of maximizing the performance of *C* but simultaneously preventing *A* to identify the alternative signal models.
Schematically, the loss function we want to minimize is defined as follows:

<img src="http://lviliani.web.cern.ch/lviliani/FiguresForDANotebook/loss.png" width="400" height="400" />



We start by importing the relevant packages.

In [ ]:
import tensorflow as tf
import tqdm
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Activation, Dense, Dropout, InputLayer
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import initializers
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from sklearn.utils import shuffle
import copy
from tensorflow.python.client import device_lib
tf.config.run_functions_eagerly(True)

Define the `NeuralNetworkWithDomainAdaptation` class as inherting from `tf.Module`. Define an `__init__` function that initializes the relevant class variables and calls the `build` function.

In the `build` function we define the structure of the model and we return the model weigths.

We define 3 Keras Sequential models:
- `model1` is the classifier network without the last output layer, i.e. the second-to-last layer is a representation of the input features;
- `model2` is the adversary network, connected to the second-to-last layer of `model1`;
- `model3` is just the output layer of the classifier that applies a softmax activation.

In the `fit` function we implement the epochs loop and we keep track of the classifier, adversary and total loss functions.

Note that we implemented a Batch Gradient Descent approach (gradients computed and weights updated every epoch using the full data), in which we call the `_train` function every epoch. A Mini-batch Gradient Descent approach can be easily implemented by splitting the training data in smaller batches.

The `_train` function contains the custom training strategy, implementing the C and A loss functions using the Categorical Cross Entropy Keras loss. In this function we compute the gradients of the loss functions and update the weights.

Take your time to look carefully at the loss definition and understand the gradients. Note that:

- we first compute the gradient of the total loss L(C+A) with respect to the classifier weights;
- then we compute the gradient of the adversary loss L(A) with respect to the adversary weights;

In [ ]:
class NeuralNetworkWithDomainAdaptation ( tf.Module ):
    
    def __init__ (self, nEpochs, learning_rate1, learning_rate2, alpha, N_NODES, n_layers1, n_layers2, n_features, n_outputsC=3, n_outputsA=7):
        self.learning_rate1 = learning_rate1
        self.learning_rate2 = learning_rate2
        self.optimizer  = tf.optimizers.Adam (self.learning_rate1)
        self.optimizer2 = tf.optimizers.Adam (self.learning_rate2)
        self.nEpochs = nEpochs
        self.alpha = alpha
        self.N_NODES = N_NODES
        self.n_layers1 = n_layers1
        self.n_layers2 = n_layers2
        self.n_features = n_features
        self.n_outputsC = n_outputsC
        self.n_outputsA = n_outputsA
        self.weightsC, self.weightsA = self.build (self.n_features, self.N_NODES)

    # Define the structure of the model
    def build (self, n_input, N_NODES):
        # initializer = initializers.Ones()

        # Feature representation (i.e. classifier w/o the last layer)
        self.model1 = Sequential()
        self.model1.add(Dense (self.N_NODES, activation = 'relu', input_dim  = n_input))
        for i in range(self.n_layers1):
            self.model1.add(Dense (self.N_NODES, activation = 'relu'))

        # Adversary model
        
        ######################
        ### YOUR CODE HERE ###
        ######################
        ### HINT: implement the adversary model (model2) similarly to what was done for model1
        ### In this case the input will be the last layer of the feature extractor.
        ### Also, you should add one last layer where a softmax activation is applied.

        # Classifier output (just the last layer of C)
        self.model3 = Sequential()
        self.model3.add(Dense (self.n_outputsC, activation = 'softmax',input_dim = self.N_NODES))      
        
        return (self.model1.weights + self.model3.weights, self.model2.weights)

    # Performs the epochs loop and the actual training.
    # Monitors the training and validation loss functions, both for the classifier and the adversary.
    # Returns the classifier categorical accuracy.
    def fit (self, X, Y, Y_adv, X_val, Y_val, Y_adv_val, show_loss = False):
        # The following lists will contain the values of the loss functions in each epoch
        losses = [] # overall loss container
        losses_adv = [] # adversary loss container
        losses_cl = [] # classifier loss container

        # Same for the "validation" losses
        losses_val = []
        losses_adv_val = []
        losses_cl_val = []

        # Compute means and sigmas for the input feature pre-processing
        self.means = np.mean ( X, axis = 0)
        self.sigmas = np.std ( X, axis = 0)

        ######################
        ### YOUR CODE HERE ###
        ######################
        ### HINT: implement the loop over the number of epochs (self.nEpochs) by hand
        ### In each iteration call the self._train function (see below) and store the loss values in the containers

        losses = np.array(losses)
        losses_adv = np.array(losses_adv)
        losses_cl = np.array(losses_cl)
               
        losses_val = np.array(losses_val)
        losses_adv_val = np.array(losses_adv_val)
        losses_cl_val = np.array(losses_cl_val)
               
        plt.plot (losses_adv, color = "y", label='Training set')
        plt.plot (losses_adv_val, color ='orange', label = "Validation set")
        plt.xlabel ("Epoch"); plt.ylabel ("Loss(A)")
        plt.legend(frameon=False)
        plt.show()

        plt.plot (losses_cl, color='r', label='Training set')
        plt.plot (losses_cl_val, color='darkred', label='Validation set')
        plt.xlabel ("Epoch"); plt.ylabel ("Loss(C)")
        plt.legend(frameon=False)
        plt.show()
        
        plt.plot (losses, color='c', label='Training set')
        plt.plot (losses_val, color='tab:blue', label='Validation set')
        plt.xlabel ("Epoch"); plt.ylabel ("Loss(C+A)")
        plt.legend(frameon=False)
        plt.show()
        
        ca = tf.keras.metrics.CategoricalAccuracy()
        ca.update_state(Y, self.predict_proba(X))
        
        return ca.result().numpy()

    # Applies a pre-processing to the input features and returns the classifier representation.
    @tf.function
    def representation (self, X):
        ppX = (X - self.means)/self.sigmas
        return  self.model1 (ppX) 

    # Returns the classifier output
    #@tf.function
    def predict_proba (self, X):
        ppX2 = self.representation (X)
        return tf.clip_by_value ( self.model3 (ppX2), 1e-7, 1. - 1e-7 )

    # Returns the adversary output
    #@tf.function
    def predict_proba_adv (self, X):
        ppX2 = self.representation (X)
        return tf.clip_by_value ( self.model2 (ppX2), 1e-7, 1. - 1e-7 )

    @tf.function
    def _train (self, X, Y, Y_adv, X_val, Y_val, Y_adv_val):
        Y_true = tf.cast (Y, tf.float32)
        Y_true_val = tf.cast (Y_val, tf.float32)
        
        Y_true_adv = tf.cast (Y_adv, tf.float32)
        Y_true_adv_val = tf.cast (Y_adv_val, tf.float32)

        with tf.GradientTape() as gt, tf.GradientTape() as gt_adv:
            #gt.watch ( self.weightsC )
            Y_hat = self.predict_proba (X) #N3(N1(x))
            Y_hat_adv = self.predict_proba_adv (X)
            Y_hat_val = self.predict_proba (X_val) #N3(N1(x)) validation set
            Y_hat_adv_val = self.predict_proba_adv (X_val)
            
            ## Training set
            # Use the categorical cross-entropy as loss function for the adversary
            cce_adv = tf.keras.losses.CategoricalCrossentropy()
            
            # Sums the contents of each row of the Y_true_adv tensor to get a 1D tensor with N_events length
            # containing 0 if the event is not VBF or 1 if it is VBF (SM or BSM)
            sum_adv = tf.math.reduce_sum(Y_true_adv, axis=1)
            
            # The second term in the multiplication is used to set the loss at 0 for non-VBF events
            # i.e. the loss of the adversary is computed only for VBF (SM and BSM) events
            loss_adv = tf.math.reduce_sum(( cce_adv( Y_true_adv, Y_hat_adv) )*( tf.cast( sum_adv.numpy()!=0, tf.float32 ) ))
            
            # Divide the loss for the total number of VBF (SM and BSM) events to get the average loss
            loss_adv = loss_adv / tf.cast((tf.math.reduce_sum(sum_adv, axis=0)).numpy(),tf.float32 )
            
            # Use the categorical cross-entropy as loss function for the classifier
            ######################
            ### YOUR CODE HERE ###
            ######################
            
            # The overall loss is L(C) - self.alpha*L(A)
            ######################
            ### YOUR CODE HERE ###
            ######################
            
            ## Validation set
            
            ######################
            ### YOUR CODE HERE ###
            ######################
            ### HINT: repeat the procedure above for the validation dataset
            
            # Compute the gradient of the overall loss with respect to the classifier weights
            gradients = ### YOUR CODE HERE ### HINT: use the tf.GradientTape gradient function

            # Then compute the gradient of L(A) with respect to the adversary weights
            gradients_adv = ### YOUR CODE HERE ### HINT: use the tf.GradientTape gradient function

        # Apply the gradients
        self.optimizer.apply_gradients ( zip(gradients, self.weightsC) )
        self.optimizer2.apply_gradients ( zip(gradients_adv, self.weightsA) )
        
        return loss, loss_adv, loss_cl, loss_val, loss_adv_val, loss_cl_val

    # Some ancillary functions to save the model weights, reset the optimizers or set specific parameters.
    def save_weights(self, model_name):
        self.model1.save_weights(model_name+'_weights_1')
        self.model2.save_weights(model_name+'_weights_2')
        self.model3.save_weights(model_name+'_weights_3')
    
    def load_weights(self, model_name):
        self.model1.load_weights(model_name+'_weights_1')
        self.model2.load_weights(model_name+'_weights_2')
        self.model3.load_weights(model_name+'_weights_3')
        
    def save_model(self, model_name):
        self.model1.save("saved_models/"+model_name+"_1")
        self.model2.save("saved_models/"+model_name+"_2")
        self.model3.save("saved_models/"+model_name+"_3")

    def reset_optimizers(self):
        self.optimizer  = tf.optimizers.Adam (self.learning_rate1)
        self.optimizer2 = tf.optimizers.Adam (self.learning_rate2)
        
    def set_alpha(self, alpha):
        self.alpha = alpha
    
    def set_epochs(self, epochs):
        self.nEpochs = epochs

# Training of the ADNN model

Now that we have defined the model structure, let's do the actual training.

## Extract the input features and labels from the original dataset and split in training (80%) and validation (20%)

First we split our data in 2 randomly chosen subsets: we will use the first one ($80\%$ of the events) as training data, while the remaining smaller subset ($20\%$ of the events) will be used for the validation.

In [ ]:
NDIM = len(features)

# Before doing the splitting, a random shuffle of the dataset doesn't hurt
df = shuffle(df)

# Perform the splitting and define training and validation datasets
msk = np.random.rand(len(df)) < 0.8
df_train = df[msk]
df_val = df[~msk]

X = df_train.values[:,0:NDIM]
Y = df_train.values[:,NDIM:NDIM+3] # isVBF, isGGH, isBKG
Y_adv = df_train.values[:,NDIM+3:NDIM+10] # isSM, isBSM0, isBSM1, ..., isBSM5

X_val = df_val.values[:,0:NDIM]
Y_val = df_val.values[:,NDIM:NDIM+3] # isVBF, isGGH, isBKG
Y_adv_val = df_val.values[:,NDIM+3:NDIM+10] # isSM, isBSM0, isBSM1, ..., isBSM5


## Build the ADNN model and define the needed parameters

Let's instantiate our `NeuralNetworkWithDomainAdaptation` class with some standard hyperparameter values.

Also, before starting the training, we want to save the initial model weights that we will reuse later in the optimization to restart every time from the same starting point.

In [ ]:
# nEpochs, learning_rate1, learning_rate2, alpha, N_NODES, n_layers1, n_layers2, n_features
adnn = NeuralNetworkWithDomainAdaptation(500, learning_rate1=0.0001, learning_rate2=0.0001, alpha=0.1, N_NODES=50, n_layers1=5, n_layers2=5, n_features=X.shape[1])

# Save initial set of weights (before training) to re-initialize the ADNN in later steps.
# Useful if we want to restart always from the same starting point during the optimization studies.
adnn.save_weights("my_model_init")

## Perform the training

And now we start the training by calling the function `fit` with the appropriate parameters.

What happens if you change the values of some hyperparameters and repeat the training? For example you can change the number of epochs with the `set_epochs(N)` method, or the learning rates of the classifier and adversary.

Don't touch the $\alpha$ parameter for now, we will see its impact later.

**YOUR TURN: train the ADNN model. Try to repeat the training changing the default value of some parameters.**

In [ ]:
### YOUR CODE HERE

# How do we quantify the ADNN performance

Now that our model is trained we want to know how well it will perform on a test dataset that has not been used to train it. But we will look also at the training dataset itself to search for signs of overtraining.

In the following we will look at several distributions and plots that will help us to quantify the ADNN performance.

## Plot the probability distributions for the three output nodes of C

First, we can look at the output distributions of the classifier. Remember that we have used a *softmax* activation function for the last layer, therefore the 3 outputs are forced to sum to unity and can be interpreted as probabilities.

In [ ]:
Y_predict_train = adnn.predict_proba(X)
Y_predict_val = adnn.predict_proba(X_val)

axis = np.linspace(0,1,20)

plt.hist(Y_predict_train[:,0].numpy(), bins = axis, label = 'VBF class - training',  histtype='step', color='r',  density=True, linewidth=2 )
plt.hist(Y_predict_train[:,1].numpy(), bins = axis, label = 'ggH class - training',  histtype='step', color='g',  density=True, linewidth=2 )
plt.hist(Y_predict_train[:,2].numpy(), bins = axis, label = 'BKG class - training',  histtype='step', color='b',  density=True, linewidth=2 )
plt.hist(Y_predict_val[:,0].numpy(), bins = axis, label = 'VBF class - validation',  histtype='step', color='r',  density=True, linewidth=2, linestyle="dashed" )
plt.hist(Y_predict_val[:,1].numpy(), bins = axis, label = 'ggH class - validation',  histtype='step', color='g',  density=True, linewidth=2, linestyle="dashed" )
plt.hist(Y_predict_val[:,2].numpy(), bins = axis, label = 'BKG class - validation',  histtype='step', color='b',  density=True, linewidth=2, linestyle="dashed" )

plt.xlabel("Probability")
plt.legend(frameon=False, bbox_to_anchor=(1.05, 1.))
plt.show()

## Plot the probability distributions for labelled events

We can now look at the probability distributions of the 3 output nodes using the known labels (`isVBF`, `isGGH`, `isBKG`) of our events.

**YOUR TURN: make the plots**

In [ ]:
### YOUR CODE HERE
### HINT: you can use the following sintax
### adnn.predict_proba( df_val[ df_val['isVBF']==1 ].values[:,0:NDIM] )[:,0]
###                                      ^                    ^           ^
###                                      |                    |           |
###                                select VBF events          |           |
###                                               get the input features  |
###                                                             select the VBF output node (i.e. node 0)

## Plot the probability distributions of different VBF signal models

If the output of the ADNN was model independent, we would expect to see statistically consistent distributions for the different VBF models. Let's then compare the VBF output node distributions for all our models.

**YOUR TURN: make the same plots as above, but this time for the `isSM`, `isBMS0`, ...,`isBSM5` labels.**

In [ ]:
def plot_model_dists():
    ### YOUR CODE HERE
    ### it's useful to embed this plots in a function, as we will reuse them later quite often

plot_model_dists()

## Plot the confusion matrices

Confusion matrices are useful tools to check in a visual way the classification power of our network. The following helper function can be used to make some nice plots.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools
#from matplotlib.backends.backend_pdf import PdfPages

def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, fontsize = 16)
    plt.yticks(tick_marks, classes, fontsize = 16)

    thresh = cm.max() / 1.2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black",fontsize=10)

    plt.xlabel("Predicted label", fontsize=16)
    plt.ylabel("True label", fontsize=16)

    
    plt.tight_layout()


We want to plot the classifier response matrix first.

**YOUR TURN: plot the classifier confusion matrix using the validation dataset**

In [ ]:
Y_pred_for_cm = adnn.predict_proba(X_val)

# Note the argmax function below, why do we need it?
Y_true_max = np.argmax(Y_val, axis=1)
Y_pred_max = np.argmax(Y_pred_for_cm, axis=1)

print('Confusion Matrix Classifier')
C = ### YOUR CODE HERE
### HINT: the easiest way is to use the confusion_matrix method we imported from sklearn.metrics
C = np.around(C, decimals=3)
print(C)

print('Classification Report')
target_names = ['VBF','ggH','BKG']
print(classification_report(Y_true_max, Y_pred_max, target_names=target_names))

plt.figure()
plot_confusion_matrix(C, classes=target_names, title='')
plt.savefig('cm.pdf', dpi='figure', bbox_inches='tight',transparent=True)

plt.show(1)

**YOUR TURN: Let's now plot the confusion matrix of the adversary**

In [ ]:
# Select a subset of the validation dataset containing only VBF (SM or BSM) events
X_val_for_cm = df_val[df_val["isVBF"]==1].values[:,0:NDIM]
Y_true_for_cm_adv = df_val[df_val["isVBF"]==1].values[:,NDIM+3:NDIM+10]
Y_pred_for_cm_adv = adnn.predict_proba_adv(X_val_for_cm)

### YOUR CODE HERE
### HINT: calculate the confusion matrix of the adversary using the same strategy as for the classifier

C2 = np.around(C2, decimals=2)

## The one below is an alternative way to compute the confusion matrix by hand
## summing the predicted values in each matrix cell for all the events and 
## dividing by the sum of the values of a given row.

# C2 = np.ndarray(shape=(tf.shape(Y_pred_for_cm_adv)[1].numpy(), tf.shape(Y_pred_for_cm_adv)[1].numpy()))

# for ev in range(tf.shape(Y_pred_for_cm_adv)[0].numpy()):
#     for pred_label in range(C2.shape[1]):
#         C2[pred_label,Y_true_max_adv[ev]] += Y_pred_for_cm_adv[pred_label,Y_true_max_adv[ev]]

# C2 = np.around( C2/C2.sum(axis=1)[:,None],decimals=2)

target_names = ['model'+str(i) for i in range(7)]

plt.figure()
plot_confusion_matrix(C2, classes=target_names, title='')
plt.savefig('cm.pdf', dpi='figure', bbox_inches='tight',transparent=True)

plt.show(1)

**YOUR TURN: a question now, how does the adversary confusion matrix look like? Can you explain why it is like that?**

## Quantitative metrics to evaluate the ADNN performance

Confusion matrices are nice looking, but let's now identify some possible quantitative metrics to assess the ADNN performance that can also be used later during the optimization.

In particular we will see the *categorical accuracy* score as a metric to quantify the overall classification performance, and two alternatives to evaluate the performance of the adversary.

In particular we can investigate two statistical tests:
- the 2-sample Kolmogorov-Smirnov test;
- the k-sample Anderson-Darling test.

In [ ]:
from sklearn.metrics import accuracy_score
from scipy.stats import ks_2samp, anderson_ksamp

# The classifier categorical accuracy is a good and simple metrics to evaluate the overall classification performance
print("Classifier categorical accuracy = ", accuracy_score(Y_true_max, Y_pred_max))
# We don't care about the adversary accuracy actually... but still here it is
print("Adversary categorical accuracy = ", accuracy_score(Y_true_max_adv, Y_pred_max_adv))

# But how do we evaluate the performance of the adversary? 
# Remember that we want the classifier output distribution to be ~ the same for different signal models.
# Let's try to compute a 2-sample Kolmogorov-Smirnov test on each pair of different models.
# The larger (smaller) is the p-value (test statistic) the higher is the compatibility between the distributions.
preds = {}
preds['isSM'] = adnn.predict_proba( df_val[ df_val['isSM']==1 ].values[:,0:NDIM] )[:,0]
for i in range(6):
    preds['isBSM'+str(i)] = adnn.predict_proba( df_val[ df_val['isBSM'+str(i)]==1 ].values[:,0:NDIM] )[:,0]
    
for i in range(7):
    for j in range(i,6):
        label1 = 'isSM' if i==0 else 'isBSM'+str(i-1)
        label2 = 'isBSM'+str(j)
        print ( "KS between %s and %s --> " %(label1,label2),ks_2samp(preds[label1],preds[label2]) )

# An interesting alternative is the k-sample Anderson-Darling test.
# It tests the null hypothesis that k-samples are drawn from the same population without having to specify the distribution function of that population. 
samples_for_ad = np.array( (preds['isSM'].numpy(), preds['isBSM0'].numpy(), preds['isBSM1'].numpy(), 
                         preds['isBSM2'].numpy(), preds['isBSM3'].numpy(), preds['isBSM4'].numpy(),
                         preds['isBSM5'].numpy()) ).T

ad_stat, _, _ = anderson_ksamp(samples_for_ad)
print(ad_stat)

# Check the impact of the $\alpha$ hyperparameter

As you may have realized, the $\alpha$ parameter has a fundamental role in this procedure and has to be tuned with care in order to achieve the desired behavior.
Let's see what happens if we set $\alpha$ by hand to some arbitrary values.

## $\alpha = 0$ - no domain adaptation

The first case we may want to study is when $\alpha = 0$. This situation is essentially the same as dropping the adversary term, given that the adversary loss has no impact (it is multiplied by 0) on the overall loss.

But note that some level of domain adaptation is still there, given that we are training our network on a dataset that is composed of different VBF signal models in equal proportions.

To avoid any bias in the training procedure, we reset the optimizers initial state with the `reset_optimizers()` helper function, as well as the initial values of the model weights (`load_weights("my_model_init")`).

**YOUR TURN: train the ADNN setting $\alpha=0$**

In [ ]:
### YOUR CODE HERE

In [ ]:
# Let's save the model weights of this configuration
adnn.save_weights("my_model_alpha0")

**YOUR TURN: now you can check the metrics we have seen before (for example the classifier categorical accuracy and the K-S test statistics) and plot the VBF output distributions for all the models.**

In [ ]:
### YOUR CODE HERE

## $\alpha = 0.01$ - let's start increasing gradually the adversary importance

**YOUR TURN: repeat the previous study but this time setting $\alpha=0.01$. What are the differences?**

In [ ]:
### YOUR CODE HERE

## $\alpha = 0.1$ - let's start increasing gradually the adversary importance

**YOUR TURN: repeat the previous study but this time setting $\alpha=0.1$. What are the differences?**

In [ ]:
### YOUR CODE HERE

## $\alpha = 1$ - let's start increasing gradually the adversary importance

**YOUR TURN: repeat the previous study but this time setting $\alpha=1$. What are the differences?**

In [ ]:
### YOUR CODE HERE

## $\alpha = 100$ - let's start increasing gradually the adversary importance

**YOUR TURN: repeat the previous study but this time setting $\alpha=100$. What are the differences?**

In [ ]:
### YOUR CODE HERE

# Tradeoff between accuracy and model dependence... How do we find a sweet spot?

We have seen that the choice of $\alpha$ is extremely important. The best value is a tradeoff between the accuracy of the classifier and the amount of model dependence.

We can use the **Optuna** tool to find a sweet spot according to the metrics we have seen previously.

In [ ]:
#!pip install optuna

We have to define a function `evaluate_performance(trial)` that does the following:

- resets the ADNN optimizers, loads the initial weights, suggests an $\alpha$ value and trains the ADNN;
- returns the K-S test statistics and the classifier categorical accuracy.

Note: Optuna makes use of various sampling strategies to suggest the hyperparameter values at each trial. The default one is a Bayesian optimization algorithm called TPE. More details can be found in the Optuna [documentation](https://optuna.readthedocs.io/en/stable/reference/samplers/generated/optuna.samplers.BaseSampler.html).

**YOUR TURN: use the template function below to implement these functionalities**

In [ ]:
import optuna

def evaluate_performance (trial) :
     # For each trial (i.e. each call of this function), suggests a float value for alpha in the defined range
    alpha = trial.suggest_float ('alpha', 0.1, 100)
    
    ### YOUR CODE HERE

    return ks_test_stat, ca

In [ ]:
# The following is needed to start the optimization
# Note that we specify 2 "directions" corresponding to whether we want to minimize or maximize the objectives
# In this case we simultaneously minimize the K-S test statistics and maximize the categorical accuracy
study = optuna.create_study(directions=['minimize', 'maximize'])
# Define the number of trials
study.optimize(evaluate_performance,n_trials=100)

Once the optimization is over, we can produce a nice [Pareto front plot](https://optuna.readthedocs.io/en/stable/reference/visualization/generated/optuna.visualization.plot_pareto_front.html) and extract a dataframe containing the best trials.

In [ ]:
optuna.visualization.matplotlib.plot_pareto_front(study, target_names=["Average K-S test", "Accuracy"])

plt.title(" ")
plt.xlabel("Average K-S test", fontsize=12)
plt.ylabel("Accuracy", fontsize=12)
plt.legend(prop={'size': 12},frameon=False)
plt.show()

opt_df = study.trials_dataframe()

for best_trial in study.best_trials:
    
    print (opt_df[opt_df["number"]==best_trial.number][['values_0','values_1','params_alpha']])
    
# Reset matplotlib default style that gets overwritten by optuna
mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams['figure.dpi'] = 150

## Let's pick the $\alpha$ value corresponding to the best trial with the lowest mean K-S and train again

**YOUR TURN: now you should be expert enough to do this with your eyes closed :). You can also consider to increase the number of epochs for this final training.**

In [ ]:
### YOUR CODE HERE

**YOUR TURN: And now check again the metrics we have seen before (for example the classifier categorical accuracy and the K-S test statistics) and plot the VBF output distributions for all the models.**

In [ ]:
### YOUR CODE HERE

# Summary

Let's finish summarizing the results. Perhaps you didn't realize, but we have actually analyzed two cases of Domain Adaptation:

1. a full-fledged Domain Adversarial Deep Neural Network;
2. a simpler approach in which we set $\alpha=0$. Note that this is still a Domain Adaptation approach (even without the adversarial term), because we trained the model on a dataset that contains events from both the source domain (SM VBF) and the target domain (BSM VBF). Also, note that in typical HEP applications the model is trained using only SM events.
3. Wait... but we haven't checked what happens in the typical application. As an excercise, let's train a simple DNN with the typical approach and compare with the previous two.

Let's run approach 3 in a separate [notebook](./Excercise_DA_MLhackathon_SimpleDNN_ForStudents.ipynb).

The `summary()` function below will output several useful information:
- the average K-S test statistic;
- the classifier categorical accuracy for the mixture of SM and BSM signal events;
- the classifier categorical accuracy for each of the 7 signal models;
- the confusion matrix of each model separately (note that the ones seen before corresponded to the SM-BSM mixture).

In [ ]:
def summary():
    preds = {}
    preds['isSM'] = adnn.predict_proba( df_val[ df_val['isSM']==1 ].values[:,0:NDIM] )[:,0]
    for i in range(6):
        preds['isBSM'+str(i)] = adnn.predict_proba( df_val[ df_val['isBSM'+str(i)]==1 ].values[:,0:NDIM] )[:,0]

    ks_test_stat = 0
    comb=0
    for i in range(7):
        for j in range(i,6):
            label1 = 'isSM' if i==0 else 'isBSM'+str(i-1)
            label2 = 'isBSM'+str(j)
            stat, pval = ks_2samp(preds[label1],preds[label2])
            ks_test_stat += stat
            comb+=1
    ks_test_stat = ks_test_stat/comb
    print("Average K-S test stat = ", ks_test_stat)

    X      = df_val.values[:,0:NDIM]
    Y_true = df_val.values[:,NDIM:NDIM+3]

    Y_pred = adnn.predict_proba(X)

    Y_true_max = np.argmax(Y_true, axis=1)
    Y_pred_max = np.argmax(Y_pred, axis=1)
    print("Classifier categorical accuracy for SM-BSM model mixture = %s" %(accuracy_score(Y_true_max, Y_pred_max)))

    #And now the categorical accuracy for each of the 7 models separately
    for i in range(7):
        label = 'isSM' if i==0 else 'isBSM'+str(i-1)
        X      = df_val[ (df_val[label]==1) | (df_val['isBKG']==1) | (df_val['isGGH']==1) ].values[:,0:NDIM]
        Y_true = df_val[ (df_val[label]==1) | (df_val['isBKG']==1) | (df_val['isGGH']==1) ].values[:,NDIM:NDIM+3]

        Y_pred = adnn.predict_proba(X)

        Y_true_max = np.argmax(Y_true, axis=1)
        Y_pred_max = np.argmax(Y_pred, axis=1)
        print("Classifier categorical accuracy for model %s = %s" %(label, accuracy_score(Y_true_max, Y_pred_max)))

        C = confusion_matrix(Y_true_max, Y_pred_max, normalize="true")
        C = np.around(C, decimals=3)
        print("Fraction of %s events classified as VBF = %s" %(label, C[0,0]))
 
        target_names = [label,'ggH','BKG']
        plt.figure()
        plot_confusion_matrix(C, classes=target_names, title='')
        plt.show(1) 

**YOUR TURN: load the ADNN weights (`adnn.load_weights(model_name)`) corresponding to the final configuration (approach 1) and to the $\alpha=0$ case (approach 2). Run the `summary()` function and compare the results. Compare also with the results obtained in the other notebook, corresponding to approach 3.**

In [ ]:
### YOUR CODE HERE

# Conclusions

In summary, comparing the 3 approaches we have seen that:

1. The ADNN performance is characterized by a low average K-S test statistic and uniform categorical accuracies for all the models.
2. The ADNN with $\alpha=0$ seems to perform better if we only look at the categorical accuracies, but the average KS is fairly high and if we look at the confusion matrices in detail we can observe that there is a large variation in the fraction of signal events classified as VBF depending on the signal model.
3. The simple DNN has a fairly high accuracy if evaluated for SM events (i.e. target domain equal to the source domain), but starts to fail dramatically as soon as we evaluate it on one of the BSM models.

# References

- Camaiani B. et al., *Model independent measurements of standard model cross sections with domain adaptation*, EPJ C 82 921 (2022), [doi:10.1140/epjc/s10052-022-10871-3](https://doi.org/10.1140/epjc/s10052-022-10871-3)
- Ben-David S., Blitzer J., Crammer K. et al. *A theory of learning from different domains*. Mach. Learn. 79, 151–175 (2010). https://doi.org/10.1007/s10994-009-5152-4
- Louppe G., Kagan M., Cranmer K. *Learning to Pivot with Adversarial Networks*, [arXiv:1611.01046](https://arxiv.org/abs/1611.01046)
- Y. Ganin et al. *Domain-adversarial training of neural networks*. J. Mach. Learn. Res. 17, 2030–2096 (2016). [arXiv:1505.07818](https://arxiv.org/abs/1505.07818)

# BACKUP (Additional studies)

## Optimize other hyperparameters

Optuna is a very powerful tool for hyperparameter optimization. In fact we can also optimize more hyperparameters simultaneously.

Similarly to what we have done with $\alpha$, let's try to optimize also something else, such as the number of layers in C and A, the number of nodes, or the learning rates.

## Optimize using the AD test

Repeat the optimization using the k-sample Anderson-Darling test instead of K-S.

## Agnosticism against unseen physics models

We may argue that the signal model chosen by Nature is unknown and in general different from the ones the discriminator has been trained on. This is indeed the primary reason why one wants the discriminator performance to be model independent in the first place. It is thus important to establish a procedure to evaluate the degree of model independence of the discriminator with respect to signal models unseen in the training.

In this specific use case, we want to check the performance of the *ADNN* for a given model that was not seen in the training. For achieving this goal, let's retrain the *ADNN* dropping one model (say `isBSM5`) from the dataset.

**NB** The results of this test are peculiar of the particular physics models taken into account.